In [1]:
import deepchem as dc
import tensorflow as tf
import os
import sys
import h5py

import numpy as np
import pandas as pd

# import matplotlib
# import matplotlib.pyplot as plt
# import matplotlib.tri 
# import mpl_toolkits.mplot3d
# My conda does not recognise this ???

import rdkit
import rdkit.Chem
from rdkit import Chem
from rdkit.Chem import Draw
import rdkit.Chem.AllChem as Chem
import rdkit.Chem.AllChem as AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors

from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.pipeline import make_union, Pipeline
from collections import Counter

from gtda.mapper import plot_interactive_mapper_graph
from gtda.plotting import plot_point_cloud, plot_betti_curves
from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_diagram
from gtda.diagrams import PersistenceEntropy
from gtda.diagrams import NumberOfPoints
from gtda.diagrams import Amplitude

import structures as st # My own module

In [3]:
def generate_structure_from_smiles(smiles): #This works for rdkit (organic materials), not really crystal structures
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)

    status = AllChem.EmbedMolecule(mol)
    status = AllChem.UFFOptimizeMolecule(mol)

    conformer = mol.GetConformer()
    coordinates = conformer.GetPositions()
    coordinates = np.array(coordinates)

    return coordinates 

def smiles_to_persistence_diagrams(smiles):
    coords=generate_structure_from_smiles(smiles)
    # Track connected components, loops, and voids
    homology_dimensions = [0, 1, 2]

    # Look into this?
    persistence = VietorisRipsPersistence(
        metric="euclidean",
        homology_dimensions=homology_dimensions,
        n_jobs=1,
        collapse_edges=True,
    )
    reshaped_coords=coords[None, :, :]
    diagrams_basic = persistence.fit_transform(reshaped_coords)  #error occurs here when the njobs > 1
    return coords, diagrams_basic

In [4]:
def persistence_diagrams(coords):
    # Track connected components, loops, and voids
    homology_dimensions = [0, 1, 2]

    # Collapse edges to speed up H2 persistence calculation!
    persistence = VietorisRipsPersistence(
        metric="euclidean",
        homology_dimensions=homology_dimensions,
        n_jobs=1,
        collapse_edges=True,
    )
    reshaped_coords=coords[None, :, :]
    diagrams_basic = persistence.fit_transform(reshaped_coords)  ###Why does this throw an error? - show to ella?
    return coords, diagrams_basic

In [5]:
# Generating the vertices of a 3x3x3 cube
nacl_structure = FCC_Structure((5.59/2), 4)
nacl_structure = np.array(nacl_structure)

coords, diagrams_basic = persistence_diagrams(nacl_structure)
plot_point_cloud(coords)

In [36]:
# I need to change these diagrams to be labelled instead of just there -- so i need to look into giotto tda and see what they're plotting and make it myself a bit here...........

nacl_structure = FCC_Structure((5.59/2), 3)
nacl_structure = np.array(nacl_structure)

coords, diagrams_basic = persistence_diagrams(nacl_structure)
plot_point_cloud(coords)
plot_diagram(diagrams_basic[0])

In [34]:
# Fix this bruhhh ??? maybe plot in bokeh
#  import data?

benzene = rdkit.Chem.MolFromSmiles("c1ccccc1")
benzene_smiles = "c1ccccc1"
coords, diagrams_basic = smiles_to_persistence_diagrams(benzene_smiles)

x_val_a = [x for x in diagrams_basic[0][:,0]]
x_val_b = [x for x in diagrams_basic[0][:,1]]
y_val = [-x for x in range(len(x_val_a))]
col = [int(x) for x in diagrams_basic[0][:,2]]

col=['r','r','r','r','r','r','r','r','r','r','r','b','g']

# for i in range(3,0,-1):
#     plt.plot([x_val_a[-i],x_val_b[-i]],[y_val[-i],y_val[-i]], c=col[-i])
# plt.legend([r'$\beta_0$',r'$\beta_1$',r'$\beta_2$'])
# for i in range(len(x_val_a)):
#     plt.plot([x_val_a[i],x_val_b[i]],[y_val[i],y_val[i]], c=col[i])
# plt.xlabel('Filter / Angstroms')
# plt.yticks([])

import plotly.graph_objects as go

x_val_a = [1, 2, 3]
y_val = [4, 5, 6]
col = ['blue', 'green', 'red']

traces_a = []
for i in range(len(x_val_a)):
    traces_a.append(go.Line(x=[x_val_a[i], x_val_a[i]], y=[y_val[i], y_val[i]], line_color=col[i]))

traces_b = []
for i in range(3, 0, -1):
    traces_b.append(go.Line(x=[x_val_a[-i], x_val_b[-i]], y=[y_val[-i], y_val[-i]], line_color=col[-i]))

fig = go.Figure(data=[*traces_a, *traces_b])

fig.update_layout(xaxis_title='Filter / Angstroms', yaxis_title='', legend_title='Legend')

fig.show()

c:\Users\fayat\anaconda3\envs\tdaf-tf2p7\lib\site-packages\plotly\graph_objs\_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


